In [57]:
import re
import numpy as np
import pickle
from math import floor
from os import listdir
from numpy.linalg import inv
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
# from sklearn.utils.extmath import randomized_svd
from scipy.sparse import csr_matrix, save_npz, load_npz
# import pandas as pd
# import warnings

# supress warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning, module="pandas", lineno=570)

# custom tokenizer with stemmer 
class PorterTokenizer(object):
    def __init__(self):
        self.pt = PorterStemmer()
    def __call__(self, doc):
        return [self.pt.stem(t) for t in RegexpTokenizer(r'(?u)\b\w\w+\b').tokenize(doc)]

# read each file into separate string

def read_docs(art_path):
    file_list = [name for name in listdir(art_path)]
    doc_list = []
    for file in file_list:
        with open(art_path + file, 'r') as myfile:
            doc_list.append(myfile.read())
    return file_list, doc_list

# vectorize strings using tf-idf vectorizer

def tfidf_transform(doc_list):
    tfidf_vectorizer = TfidfVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer(), smooth_idf=True)
    tfidf_trans_matrix = tfidf_vectorizer.fit_transform(doc_list)
    return tfidf_trans_matrix, list(tfidf_vectorizer.vocabulary_.keys())

# perform low rank approximation with given multiplier

def lra_mult(tfidf_matrix, mult):
    (_, n_org) = tfidf_matrix.shape
    n_comp = floor(n_org * mult)
    if n_comp < 2:
        n_comp = 2
    svd = TruncatedSVD(n_comp, "arpack")
    reduced_m = svd.fit_transform(tfidf_matrix)
    return reduced_m

def lra(tfidf_matrix, n_comp):
    if n_comp < 2:
        n_comp = 2
    svd = TruncatedSVD(n_comp, "arpack")
    reduced_m = svd.fit_transform(tfidf_matrix)
    return reduced_m

# save info

# def save_matrix(path, filename, tfidf_matrix):
#     save_npz(file=path+filename, matrix=tfidf_matrix)
    
# def save_voc(path, filename, voc):    
#     with open(path+filename, 'w+') as voc_file:
#         for item in voc:
#             voc_file.write("%s\n" % item)

def save_list(path, filename, l):
    with open(path+filename, 'w+') as f:
        for item in l:
            f.write("%s\n" % item)
            
def save_info(path, tfidf_matrix, file_list, voc, lsa, tfidf_matrix_t):
    save_npz(file=path+'tfidf_matrix', matrix=tfidf_matrix)
    with open(path+'tfidf_matrix_t', 'wb') as f:    
        np.save(f, tfidf_matrix_t)
    save_list(path, 'voc', voc)
    save_list(path, 'file_list', file_list)
    with open('lsa.pickle', 'wb') as f:    
        pickle.dump(lsa, f, pickle.HIGHEST_PROTOCOL)

# load saved info

# def load_matrix(path, matrix_filename):
#     tfidf_trans_matrix = load_npz(file=path+matrix_filename+'.npz')
#     return tfidf_trans_matrix

def load_list(path, filename):
    l = []
    with open(path+filename, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            l.append(line)
    return l

# def load_voc(path, voc_filename):
#     voc = []
#     with open(voc_filename, 'r') as myfile:
#         for line in myfile:
#             line = line.rstrip('\n')
#             voc.append(line)
#     return voc

def load_info(path, matrix_filename, file_list_filename, voc_filename, lsa_filename, matrix_t_filename):
    tfidf_matrix = load_npz(file=path+matrix_filename+'.npz')
    with open(path+matrix_t_filename, 'rb') as f:    
        tfidf_matrix_t = np.load(f)
    file_list = load_list(path, file_list_filename)
    voc = load_list(path, voc_filename)
    with open('lsa.pickle', 'rb') as f:    
        lsa = pickle.load(f)
    with open(path+matrix_t_filename, 'rb') as f:    
        tfidf_matrix_t = np.load(f)    
    return tfidf_matrix, file_list, voc, lsa, tfidf_matrix_t

# search

def search(search_str, n, tfidf_matrix, voc, file_list):
    tfidf_search_vectorizer = TfidfVectorizer(vocabulary=voc, stop_words='english', tokenizer=PorterTokenizer(), smooth_idf=True)
    tfidf_search_matrix = tfidf_search_vectorizer.fit_transform(search_str)
    sim_matrix = cosine_similarity(tfidf_matrix, tfidf_search_matrix)
    a = [i[0] for i in sorted(enumerate(list(sim_matrix[:,0])), key=lambda x:x[1], reverse=True)]
    res = [file_list[i] for i in a]
    return res[:n]

def lsa_learn(tfidf_matrix, n_comp):
    lsa = TruncatedSVD(n_components=n_comp)
    tfidf_matrix_t = lsa.fit_transform(tfidf_matrix)
    tfidf_matrix_t = Normalizer(copy=False).fit_transform(tfidf_matrix_t)
    return lsa, tfidf_matrix_t

def lsa_search(query_str, n, tfidf_matrix_t, voc, lsa, file_list):
    tfidf_query_vectorizer = TfidfVectorizer(vocabulary=voc, stop_words='english', tokenizer=PorterTokenizer(), smooth_idf=True)
    tfidf_query_matrix = tfidf_query_vectorizer.fit_transform(query_str) 
    query = lsa.transform(tfidf_query_matrix)
    sim_matrix = cosine_similarity(tfidf_matrix_t, query)
    a = [i[0] for i in sorted(enumerate(list(sim_matrix[:,0])), key=lambda x:x[1], reverse=True)]
    res = [file_list[i] for i in a]
    return res[:n]

In [3]:
art_path = r'/home/yurii/projects/mownit2/lab6/articles/'
path = r'/home/yurii/projects/mownit2/lab6/'

In [30]:
# create

n_comp = 100

file_list, doc_list = read_docs(art_path)
tfidf_matrix, voc = tfidf_transform(doc_list)
lsa, tfidf_matrix_t = lsa_learn(tfidf_matrix, n_comp)
save_info(path, tfidf_matrix, file_list, voc, lsa, tfidf_matrix_t)

In [55]:
save_info(path, tfidf_matrix, file_list, voc, lsa, tfidf_matrix_t)

In [59]:
# load

tfidf_matrix, file_list, voc, lsa, tfidf_matrix_t = load_info(path, 'tfidf_matrix', 'file_list', 'voc', 'lsa.pickle', 'tfidf_matrix_t')

In [27]:
query_str = ["hurricane"]

In [61]:
result_list = search(query_str, 5, tfidf_matrix, voc, file_list)
print(result_list)

['HMS Hood', 'Battle of Khafji', 'Ruma Maida', 'Ahalya', 'Eadbald of Kent']


In [45]:
lsa, tfidf_matrix_t = lsa_learn(tfidf_matrix, 300)

In [62]:
result_list = lsa_search(query_str, 5, tfidf_matrix_t, voc, lsa, file_list)
print(result_list)

['HMS Hood', 'Courageous-class battlecruiser', 'HMS Vanguard (23)', 'HMS Queen Mary', 'St Vincent-class battleship']


In [ ]:
################################################################################################

In [1]:
# res = lsa_search(query_str, 5, tfidf_matrix_t, voc, n_comp=200)
# print(res)

In [64]:
################################################################################################

In [73]:
# process generated files
abs_path = r'/home/yurii/projects/mownit2/lab6/outl/AB/'
cur_path = r'/home/yurii/projects/mownit2/lab6/a/'
for filename in listdir(abs_path):
    with open(abs_path + filename, 'r') as myfile:
        s = myfile.read()
        p = re.compile('<doc[^>]*>([^<]*)<\/doc>')
        l = p.findall(s)
        for i in range(0,len(l)):
            p = re.compile('Notes\n|Footnotes\n|Bibliography\n|References\n|External links\n|Further reading\n')
            l[i] = p.sub(r'', l[i])
            p = re.compile('\n{3,}')
            l[i] = p.sub(r'\n', l[i])
            if l[i][0] == '\n':
                l[i] = l[i][1:]
            title = l[i].splitlines()[0]
            title = title.replace(r'/', ' ')
            with open(cur_path + title, 'w+') as myfile2:
                myfile2.write(l[i])

In [15]:
# import nltk
# nltk.download()

In [16]:
# vector = CountVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer())
# trans_vect = vector.fit_transform(doc_list)
# dt = pd.DataFrame(trans_vect.toarray(), index=file_list, columns=vector.get_feature_names()).head(10)

In [14]:
# vector.get_feature_names()

In [29]:
# example = [
#     'In mathematics, 1 − 2 + 3 − 4 + ··· is the infinite series whose terms are the successive positive integers, given alternating signs. Using sigma summation notation the sum of the first "m" terms of the series can be expressed as',
#     'The infinite series diverges, meaning that its sequence of partial sums, , does not tend towards any finite limit. Nonetheless, in the mid-18th century, Leonhard Euler wrote what he admitted to be a paradoxical equation:',
#     "A rigorous explanation of this equation would not arrive until much later. Starting in 1890, Ernesto Cesàro, Émile Borel and others investigated well-defined methods to assign generalized sums to divergent series—including new interpretations of Euler's attempts."
# ]

# vect1 = CountVectorizer(min_df=1, stop_words='english', tokenizer=PorterTokenizer())
# # vect2 = CountVectorizer(min_df=1, stop_words='english')
# dtm1 = vect1.fit_transform(example)
# # dtm2 = vect2.fit_transform(example)

In [32]:
# pd.DataFrame(dtm1.toarray(), index=example, columns=vect1.get_feature_names()).head(10)
# vect1.get_feature_names()

In [33]:
# pd.DataFrame(dtm2.toarray(), index=example, columns=vect2.get_feature_names()).head(10)
# vect2.get_feature_names()

In [12]:
# dict_list = [col.OrderedDict([]) for i in range(0,len(file_list))]

# def read_words(words_file):
#     return [word for line in open(words_file, 'r') for word in line.split()]

# file_name = art_path + r'0'
# wl = read_words(file_name)